<a href="https://colab.research.google.com/github/yg4236/Machine-Deep_Learning/blob/main/ajou_2021_spring_ml_termproject_(1)_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from google.colab import drive

#load train data and label
drive.mount('/content/drive')
train_data_path = "/content/drive/My Drive/기계학습 1등팀/data/train_data.npy" # Your train data file
train_label_path = "/content/drive/My Drive/기계학습 1등팀/data/train_label.npy" # Your train label file

train_data = np.load(train_data_path, allow_pickle=True)
train_label_rawdata = np.load(train_label_path, allow_pickle=True)  

num_train_data = len(train_data)

Mounted at /content/drive


In [2]:
# Get last baduk data of all game
last_train_data = np.zeros( (num_train_data,502, 19,19) )

for i in range(num_train_data):
  temp_last_data = train_data[i]
  for j in range(len(temp_last_data)):
    last_train_data[i, j, :] = temp_last_data[j]

In [3]:
# Prepare label for training SVM
rating = ['18k', '17k', '16k', '15k', '14k', '13k', '12k', '11k', '10k', 
          '9k', '8k', '7k', '6k', '5k', '4k', '3k', '2k', '1k',
          '1d', '2d', '3d', '4d', '5d', '6d', '7d', '8d', '9d']

In [4]:
# Prepare label for training SVM (character rating to integer rating)
train_label = np.zeros(num_train_data)
for i in range(num_train_data):
  train_label_temp = train_label_rawdata[i]
  train_label_idx = rating.index(train_label_temp)
  train_label[i] = train_label_idx

In [ ]:
# Train SVM using training data and label (This part shoud be done by ML algorithm!) --> Key part! (학생 여러분들은 여기에 머신러닝 알고리즘을 구현하면 됩니다. 가장 중요한 부분입니다.)

In [18]:
import torch
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(502, 502, kernel_size=3,stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(502, 1004, kernel_size=3,stride=2, padding=1),
            torch.nn.ReLU())

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(1004, 1004, kernel_size=3,stride=2, padding=1),
            torch.nn.ReLU())

        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(1004, 2008, kernel_size=3,stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.AvgPool2d(kernel_size=2, stride=2))

        # self.layer5 = torch.nn.Sequential(
        #     torch.nn.Conv2d(516, 1024, kernel_size=3,stride=1, padding=1),
        #     torch.nn.ReLU(),
        #     torch.nn.MaxPool2d(kernel_size=2, stride=2))


        # 전결합층 
        self.fc = torch.nn.Linear(2008, 27, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        # print(out.shape)
        out = self.layer2(out)
        # print(out.shape)
        out = self.layer3(out)
        # print(out.shape)
        out = self.layer4(out)
        # print(out.shape)
        # out = self.layer5(out)
        # print(out.shape)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

class BadukDataset():
    def __init__(self, array, label):
        super().__init__()
        self.array = array
        self.label = label
        # self.transforms = transforms

    def __getitem__(self, index):
        final = self.array[index]
        final_label = self.label[index]
        final_label = torch.tensor(final_label,dtype=torch.long)
        return final, final_label

    def __len__(self):
        return len(self.array)

train_dataset = BadukDataset(last_train_data, train_label)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)

In [17]:
import torch.nn as nn
import torch.optim as optim
net = CNN().double()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
device = 'cuda'
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
print (device,"로 학습")

# 신경망 학습하기
for epoch in range(50):
    running_loss = 0.0
    for i, data in enumerate(trainloader,0):
        inputs, labels = data  # data입력
        optimizer.zero_grad()  # gradient 매개변수를 0으로 만듬
        # 순전파 + 역전파 + 최적화
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # 통계출력
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print('Finished Training')
PATH = '/content/drive/My Drive/기계학습 1등팀/data/baduk_net4.pth'
torch.save(net, PATH)

cuda 로 학습


KeyboardInterrupt: ignored

In [ ]:
# Get test data
test_data_path = "/content/drive/My Drive/기계학습 1등팀/data/test_data.npy"
test_data = np.load(test_data_path, allow_pickle=True)

In [ ]:
# Get last test data
num_test_data = len(test_data)

last_test_data = np.zeros( (num_test_data,1, 19, 19) )

for i in range(num_test_data):
  temp_last_data = test_data[i][-1]
  last_test_data[i, :, :] = temp_last_data


In [ ]:
# Predict test labels
ts_last_test_data = torch.tensor(last_test_data)
outputs = net(ts_last_test_data)
_, predicted =torch.max(outputs, 1)


In [ ]:
# Get test labels (student does not have this ground-truth of test data)
test_label_path = "/content/drive/My Drive/기계학습 1등팀/data/test_label2.npy"
test_label_rawdata = np.load(test_label_path, allow_pickle=True)
test_label = np.zeros(num_test_data)
for i in range(num_test_data):
  test_label_temp = test_label_rawdata[i]
  test_label_idx = rating.index(test_label_temp)
  test_label[i] = test_label_idx

In [ ]:
test_dataset = BadukDataset(ts_last_test_data, test_label)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=True, num_workers=0)
PATH = './baduk_net4.pth'
net = torch.load(PATH)
net.eval()

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 128, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (0): Conv2d(256, 516, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer5): Sequential(
    (0): Conv2d(516, 1024, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilati

In [ ]:
# 전체 데이터 셋에 대한 결과 확인
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
print('Accuracy on test data : %f ' % (correct/len(test_label)))

Accuracy on test data : 0.061338 


# 새 섹션